In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import MySQLdb as mdb
import collections 
import jieba
from snownlp import SnowNLP

## 歌曲对应评论数量

In [3]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql('SELECT song_id, count(*) FROM 163music.analysis_comment group by song_id', con=mysql_cn)    
mysql_cn.close()

In [9]:
df.sort_values(by=['count(*)'], ascending=False, inplace=True)

In [12]:
df.to_csv('song_comment_count.csv')

## 歌曲评论数

In [14]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql('SELECT DISTINCT song_id,total FROM 163music.api_commentcount', con=mysql_cn)    
mysql_cn.close()

In [26]:
df.drop_duplicates(['song_id'], inplace=True)
df.sort_values(by=['total'], ascending=False, inplace=True)

In [27]:
df.shape

(8018, 2)

In [28]:
df.to_csv('song_count.csv')

## 用户评论数

In [29]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql('SELECT user_id, count(*) FROM 163music.analysis_comment group by user_id', con=mysql_cn)    
mysql_cn.close()

In [30]:
df.shape

(2979915, 2)

In [31]:
df.drop_duplicates(['user_id'], inplace=True)
df.sort_values(by=['count(*)'], ascending=False, inplace=True)

In [32]:
df.to_csv('user_comment_count.csv')

## 单条评论点赞数

In [33]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql("SELECT DISTINCT comment_id, liked_count FROM 163music.analysis_comment where liked_count != '0'", con=mysql_cn)    
mysql_cn.close()

In [34]:
df.shape

(3988560, 2)

In [35]:
df.sort_values(by=['liked_count'], ascending=False, inplace=True)

In [36]:
df.to_csv('comment_liked_count.csv')

## 单挑评论点赞数 带有评论内容

In [37]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql("SELECT DISTINCT comment_id, content,liked_count FROM 163music.analysis_comment where liked_count != '0'", con=mysql_cn)    
mysql_cn.close()

In [38]:
df.sort_values(by=['liked_count'], ascending=False, inplace=True)

In [39]:
df.to_csv('comment_liked_count_content.csv')

## 单条评论点赞数 带有内容和歌曲信息

In [40]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql("SELECT DISTINCT comment_id, content,liked_count ,song_id FROM 163music.analysis_comment where liked_count != '0'", con=mysql_cn)    
mysql_cn.close()
df.sort_values(by=['liked_count'], ascending=False, inplace=True)
df.to_csv('comment_liked_count_content_song_id.csv')

## 用户获赞数(sum)

In [41]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql("SELECT user_id, sum(liked_count) FROM 163music.analysis_comment group by user_id", con=mysql_cn)    
mysql_cn.close()

In [42]:
df.head()

,user_id,sum(liked_count)
0,1,46306.0
1,100000086,9.0
2,100000091,5.0
3,100000093,2.0
4,100000120,8.0


In [43]:
df.sort_values(by=['sum(liked_count)'], ascending=False, inplace=True)
df.to_csv('user_liked_count_sum.csv')

## 歌曲互动用户数

In [47]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql("SELECT song_id, count(user_id) FROM 163music.analysis_comment group by song_id", con=mysql_cn)    
mysql_cn.close()

In [48]:
df.head()

,song_id,count(user_id)
0,1001054,8
1,1002595,5
2,1003708,35
3,1003956,18
4,1004236,3


In [49]:
df.sort_values(by=['count(user_id)'], ascending=False, inplace=True)
df.to_csv('song_user_count.csv')

In [27]:
mysql_cn= mdb.connect(host='localhost', port=3306,user='root', passwd=' ', db='163music',charset='utf8mb4')
df = pd.read_sql('SELECT song_id, count(*) FROM 163music.analysis_comment where user_id=310060800', con=mysql_cn, index_col='id')    
mysql_cn.close()

In [30]:
df = df.sort_values(by=['liked_count'],ascending=False)
df.head()

,song_id,content,comment_id,user_id,pub_time,liked_count
id,,,,,,


In [29]:
total_comment_count = df.shape[0]
total_liked_count = df.liked_count.sum()
total_deta_days = (df.pub_time.max()-df.pub_time.min()).days
print('在',total_deta_days,'天里共发表了',total_comment_count,'条评论获得',total_liked_count,'次点赞')

AttributeError: 'float' object has no attribute 'days'

In [26]:
most_liked_comment = df.iloc[0]
print('其中在', most_liked_comment.pub_time,'发表的',most_liked_comment.content,'获得最多',most_liked_comment.liked_count,'次点赞')

其中在 2016-07-13 15:31:35.876000 发表的 很喜欢小鸠也很喜欢这首歌。 获得最多 2 次点赞
